In [2]:
import numpy as np

At time step t, given the measurement signal   tau_t

# Testing code

In [3]:
n = 3;
n_out = 2;
n_res = 100; #size reservoir
s = np.zeros((n,1))               # From self-organized layer
r = np.zeros((n_res,1))           # From Reservoir
W_out = np.zeros((n_out,n_res))
W_dir = np.zeros((n_out,n))
W_train = np.hstack((W_out, W_dir)) # Size n_out x (n_res + n)
tau = 0;
c_t = np.hstack((r.T,s.T)).T # Size (n_res + n) x 1


# Inputs
sigma_2 = 0.01;
phi_2 = 1; #random number for now

#Initialize 
V_0 = np.zeros((n_res+n, n_res+n, n_out)) # size     (n_res+n) x (n_res+n) x (n_out)
for i in range(n_out):
    V_0[:,:,i] = phi_2 * np.eye(n_res + n)

# tau_t received from the sensor
tau_t = 5
    
# From t = 1
    
# Computation of c_t+1
# For t+1
c_dot = np.hstack((r.T,s.T)).T # Size (n_res + n) x 1
o_dot = np.dot(W_train,c_dot)

#Update the weights
V_dot = np.zeros((n_res+n, n_res+n, n_out))
#Update V
for i in range(n_out):
    V_dot[:,:,i] = np.linalg.inv(V_0[:,:,i] + (1/sigma_2)* c_t @ c_t.T)

W_train_dot = np.zeros(np.shape(W_train))
for i in range(n_out):
    a = V_dot[:,:,i] @ np.linalg.inv(V_0[:,:,i]) @ W_train[i,:].T
    b = (1/sigma_2) * V_dot[:,:,i] @ c_dot * tau
    temp = np.sum([a.T, b.T], axis=0)
    W_train_dot[i,:] = temp

# Code to implement with the rest of the network

Assuming that we create an overall class "Neural Network"

In [6]:
class output_layer:
    def __init__(self, s_size, r_size, n_ouput, sigma_2 = 0.1, phi_2 = 1):
        
        self.s_size = s_size   # Input from self-organized layer
        self.r_size = r_size   # Input from Reservoir
        self.o_size = o_size
        self.n_ouput = n_ouput
        self.sigma_2 = sigma_2
        self.phi_2 = phi_2
        #self.tau = tau_sens # Torque measurements
        
    def initweights(self):
        W_out = np.random.random((self.n_ouput,self.r_size))
        W_dir = np.random.random((self.n_ouput,self.s_size))
        #Normalise W_out, W_dir
        W_outmax, W_outmin = W_out.max(), W_out.min()
        W_dirmax, W_dirmin = W_dir.max(), W_dir.min()
        self.W_out = (W_out - W_outmin)/(W_outmax - W_outmin)
        self.W_dir = (W_dir - W_dirmin)/(W_dirmax - W_dirmin)
        self.W_train = np.hstack((W_out, W_dir))
        
        #Initialize V
        self.V = self.phi_2 * np.identity((self.r_size + self.s_size, self.r_size + self.s_size, self.n_ouput))
        
    def update_parameters(self, c_t, tau, W_train):
        #Update V
        V_cache = self.V
        for i in range(n_out):
            self.V[:,:,i] = np.linalg.inv(self.V[:,:,i] + (1/self.sigma_2)* c_t @ c_t.T)
        #Update W_train
        for i in range(n_out):
            a = self.V[:,:,i] @ np.linalg.inv(V_cache[:,:,i]) @ W_train[i,:].T
            b = (1/sigma_2) * V_dot[:,:,i] @ c_dot * tau
            self.W_train[i,:] = np.sum([a.T, b.T], axis=0)
            
    def _update(c_dot):
        o_dot = np.dot(self.W_train, c_dot)
        return o_dot
        
    def fit(self, self_input, res_input, tau_sens, outputs, epoch):    #TRAINING
        """
        Collect the network's reaction to training data, train readout weights.

        Args:
            inputs: array of dimensions (n_res + n_self) x N_training_samples
            outputs: array of dimension n_outputs x N_training_samples

        Returns:
            the network's output on the training data, using the trained weights
        """
        # Maybe it's needed to add some reshape in order to adapt the input data to the code
        
        c = np.hstack((self_input.T,res_input.T)).T # Size (n_res + n_self) x N_training_samples
        for n in range(1, c.shape[1]):
            output[n, :] = self._update(c[:,n])
            #Update the weights when given torque measurement
            if n!=1:
                update_parameters(self, c[:,n-1], self.tau[n], self.W_train)
            else:
                update_parameters(self, c_t, self.tau[n], np.zeros(W_train.shape))
                    
    def predict(self, self_input, res_input):
        """
        Apply the learned weights to the network's reactions to new input.

        Args:
            inputs: array of dimensions (N_test_samples x n_inputs)
            continuation: if True, start the network from the last training state

        Returns:
            Array of output activations
        """
        c = np.hstack((self_input.T,res_input.T)).T
        return self._update(c)

# Random Data Sets

In [7]:
n_samples = 10
n_self = 10
n_res = 10
n_out = 3
train_self_inputs = np.random.random((n_self,n_samples))
train_res_inputs = np.random.random((n_res,n_samples))
tau_sens = 1;
train_output = np.random.random((n_out,n_samples))

# Testing

In [ ]:
output_layer = output_layer(s_size = 10,
                            r_size = 10,
                            n_ouput = 3,
                            sigma_2 = 0.1,
                            phi_2 = 1)

pred_train = output_layer.fit(train_self_inputs,train_res_inputs, tau_sens, train_output)
print("test error:")
pred_test = esn.predict(test_self_inputs,test_res_inputs)
print(np.sqrt(np.mean((pred_test - test_output)**2)))
